In [1]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
hltb = requests.get('https://howlongtobeat.com/game.php?id=4250').content
hltb_soup = BeautifulSoup(hltb, "lxml")

def get_game_title(soup):
    return soup.find('div', attrs = {'class': 'profile_header shadow_text'}).get_text().strip('\n').strip(' $')

get_game_title(hltb_soup)

'Half-Life 2: Episode One'

In [3]:
def text_from_tag(tag):
    return tag.get_text().strip('\n').strip(' $')

In [4]:
def get_average_time(times):
    return text_from_tag(times[1])

def get_median_time(times):
    return text_from_tag(times[2])

def get_rushed_time(times):
    return text_from_tag(times[3])

def get_leisure_time(times):
    return text_from_tag(times[4])

def get_poll_count(times):
    return text_from_tag(times[0])

In [5]:
def get_main_story_times(soup):
    return soup.find("td", text="Main Story").find_next_siblings("td")

def get_main_story_extras_times(soup):
    return soup.find("td", text="Main + Extras").find_next_siblings("td")

def get_completionist_times(soup):
    return soup.find("td", text="Completionists").find_next_siblings("td")

def get_all_playstyles_times(soup):
    return soup.find("td", text="All PlayStyles").find_next_siblings("td")

In [38]:
def generate_row(tag, soup, play_title, id_no):
    return pd.DataFrame(data = [[get_game_title(soup), id_no, play_title, get_poll_count(tag), 
                                 get_average_time(tag), get_median_time(tag), get_rushed_time(tag), 
                                 get_leisure_time(tag)]],
                       columns = ['Title', 'ID', 'Playstyle', 'Polled', 'Average', 'Median', 'Rushed', 'Leisure'])

def cook_soup(id_no):
    return BeautifulSoup(requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content, "lxml")

def create_game_entry(id_no):
    soup = cook_soup(id_no)
    valid_page = [soup.find("td", text="Main Story"), soup.find("td", text="Main + Extras"),
                 soup.find("td", text="Completionists"), soup.find("td", text="All PlayStyles")]
    if None in valid_page:
        return pd.DataFrame()
    else:
        if soup.findAll('tr'):
            main = generate_row(get_main_story_times(soup), soup, "Main", id_no)
            main_extra = generate_row(get_main_story_extras_times(soup), soup, "Main + Extras", id_no)
            completionist = generate_row(get_completionist_times(soup), soup, "Completionist", id_no)
            all_playstyles = generate_row(get_all_playstyles_times(soup), soup, "All Playstyles", id_no)
            return main.append(main_extra).append(completionist).append(all_playstyles).set_index(['ID', 'Title',
                                                                                                   'Playstyle'])
        else:
            return pd.DataFrame()

half_life_episode_two = create_game_entry("4250")
half_life_episode_two

Polled Average Median  Rushed  \
ID   Title                    Playstyle                                      
4250 Half-Life 2: Episode One Main              805  3h 55m     4h  2h 40m   
                              Main + Extras     229  4h 28m     4h  3h 07m   
                              Completionist     190  5h 32m     5h  3h 23m   
                              All Playstyles   1.2K  4h 16m     4h  2h 46m   

                                              Leisure  
ID   Title                    Playstyle                
4250 Half-Life 2: Episode One Main             6h 58m  
                              Main + Extras    7h 44m  
                              Completionist   18h 38m  
                              All Playstyles  18h 04m

In [39]:
rdr2 = create_game_entry('27100')
rdr2

Polled   Average   Median  \
ID    Title                 Playstyle                                  
27100 Red Dead Redemption 2 Main              238   45h 50m      45h   
                            Main + Extras     500   77h 12m      72h   
                            Completionist      64  151h 18m     150h   
                            All Playstyles    802   73h 48m  66h 03m   

                                              Rushed   Leisure  
ID    Title                 Playstyle                           
27100 Red Dead Redemption 2 Main             32h 26m   64h 35m  
                            Main + Extras    52h 17m  150h 48m  
                            Completionist   110h 55m  265h 58m  
                            All Playstyles   44h 41m  225h 36m

In [42]:
foo = np.arange(172, 18999)

In [ ]:
bar = []
import time
start = time.time()

for i in foo:
    data = create_game_entry(str(i))
    #bar.append(data)
    if not data.empty:
        data.to_csv("hl2b.csv", mode='a', header=False)
        print(i)

#pd.concat(bar)

end = time.time()
print(end - start)

172
174
175
180
191
196
198
199
205
206
207
210
213
219
223
232
234
236
237
238
247
248
250
253
254
255
256
262
266
268
269
270
271
272
274
275
277
278
281
283
296
302
309
310
319
320
321
322
323
324
329
330
331
340
343
346
347
348
353
